<a href="https://colab.research.google.com/github/lalapopa/fall_guy_bot/blob/main/trainNN_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 
import time 
import random
import numpy as np 
import random

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.regularizers import l2

from tensorflow.keras.callbacks import TensorBoard


In [ ]:
path_gd = 'drive/MyDrive/LEVELS/'
keys_categories = {
'W':        [1,0,0,0,0,0,0],
'A':        [0,1,0,0,0,0,0],
'S':        [0,0,1,0,0,0,0],
'D':        [0,0,0,1,0,0,0],
'SPACE':    [0,0,0,0,1,0,0],
'E':        [0,0,0,0,0,1,0],
'SHIFT':    [0,0,0,0,0,0,1],
'NOTHING':  [0,0,0,0,0,0,0],
}

In [ ]:
level = 'KnightFever'

In [ ]:
X = np.load(open(f'{path_gd}{level}/X.npy','rb'))
y = np.load(open(f'{path_gd}{level}/y.npy','rb'))

print(X.shape)
print(y.shape)

X = X/255.0
X = X.astype(np.int16)



(25384, 120, 200, 1)
(25384, 7)


In [ ]:
def only_w(array):
    new_arr = array[1:]
    if any(new_arr):
        return False
    if array[0] == 1:
        return True

In [ ]:
nothing_counter = 0
w_counter = 0
for lable in y:
    if not any(lable):
        nothing_counter += 1
    if only_w(lable):
        w_counter += 1
        
print(nothing_counter)
print(w_counter)
        

2049
13246


In [ ]:
def get_train_and_test_data(X_array, y_array, data_percent=0.1):

    y_normalized, taken_position = normalize_data(y_array)
    X_normalized = [X_array[pos] for pos in taken_position]

    max_index = get_img_amount(X_normalized) - 1  #we need last index available 

    print(f'img_number = {max_index}')

    list_size_for_test = int(max_index*data_percent)

    index_for_test = get_unique_random_list(list_size_for_test, max_index)
    index_for_train = get_list_without_given_element(max_index, index_for_test)
    random.shuffle(index_for_train)

    print('creating X_test')
    X_test = np.stack([X_normalized[num] for num in index_for_test])

    print('creating y test')
    y_test = np.stack([y_normalized[num] for num in index_for_test])

    print('creating X_train')
    X_train = np.stack([X_normalized[num] for num in index_for_train])

    print('creating y_train')
    y_train = np.stack([y_normalized[num] for num in index_for_train])

    print('try to return')
    return X_train, y_train, X_test, y_test

def normalize_data(y_array):
    key_pos = [0, 1, 2, 3, 4, 5]  # 0 - W; 1 - A; 2 - S; 3 - D; 4 - SPACE;
    key_names = [
    'W_nums', 
    'A_nums',
    'S_nums',
    'D_nums',
    'SPACE_nums',
    'E_nums',
    'nothing'
    ] 
    press_dict = {
    'W_nums': 0,
    'A_nums': 0,
    'S_nums': 0,
    'D_nums': 0,
    'SPACE_nums': 0,
    'E_nums': 0,
    'nothing': 0,
    }

    pos_in_y = {
    'W_nums': [],
    'A_nums': [],
    'S_nums': [],
    'D_nums': [],
    'SPACE_nums': [],
    'E_nums': [],
    'nothing': [],
    }
    for lable_index, label in enumerate(y_array):
        if any(label):
            if only_w(label):
                pos_in_y['nothing'].append(lable_index)
                press_dict['nothing'] += 1 
                continue
            one_pos = find_one_in_array(label)

            if len(one_pos) == 1:
                for key in key_pos:
                    if key in one_pos:
                        pos_in_y[key_names[key]].append(lable_index)
                        press_dict[key_names[key]] += 1

            if one_pos[0] == 0 and len(one_pos) == 2:
                for key in key_pos[1:]:
                    if key in one_pos:
                        pos_in_y[key_names[key]].append(lable_index)
                        press_dict[key_names[key]] += 1

        else:
            pos_in_y['nothing'].append(lable_index)
            press_dict['nothing'] += 1 

    # press_amount = sum(press_dict.values())
    # avg_press = int(press_amount/len(press_dict))
  
    max_value = max(press_dict.values())
    print(f'press_number in key = {press_dict}')
    print(f'max press = {max_value}')
    new_pos_y = {}

    for key_name, value in pos_in_y.items():
        new_pos_y[key_name] = []
        if len(value) != 0:
            if len(value) < max_value:
                while max_value > len(new_pos_y[key_name]):
                    for i in value:
                        new_pos_y[key_name].append(i)
                        if len(new_pos_y[key_name]) >= max_value:
                            break

            if len(value) >= max_value:
                for_shuffle = [i for i in value]
                random.shuffle(for_shuffle)
                new_pos_y[key_name] = for_shuffle[:max_value]

    y_new = []
    positions = []

    #we take all key except 'W'
    new_pos_y = dict(list(new_pos_y.items())[1:]) #Removing W from dict
    for key, value in new_pos_y.items():
        print(f'In Key = {key}, press len list = {len(value)}')
        for pos in value:
            y_new.append(y_array[pos][1:]) #Removing W in list now index 0 is A key
            positions.append(pos)

    return y_new, positions

def only_w(array):
    new_arr = array[1:]
    if any(new_arr):
        return False
    if array[0] == 1:
        return True

def find_one_in_array(array):
    return [i for i, val in enumerate(array) if val]

def get_img_amount(array):
    shape = np.shape(array)
    shape_in_list = list(shape)
    return shape_in_list[0]


def get_unique_random_list(list_len, max_number):
    return random.sample(range(0, max_number), list_len)


def get_list_without_given_element(array_size, element_positions):
    array = [num for num in range(0,array_size)]
    pos_to_delete = tuple(element_positions)
    for i in pos_to_delete:
        array.remove(i)
    return array

In [ ]:
X_train, y_train, X_test, y_test = get_train_and_test_data(X,y)

press_number in key = {'W_nums': 0, 'A_nums': 3546, 'S_nums': 226, 'D_nums': 4194, 'SPACE_nums': 582, 'E_nums': 291, 'nothing': 15295}
max press = 15295
In Key = A_nums, press len list = 15295
In Key = S_nums, press len list = 15295
In Key = D_nums, press len list = 15295
In Key = SPACE_nums, press len list = 15295
In Key = E_nums, press len list = 15295
In Key = nothing, press len list = 15295
img_number = 91769
creating X_test
creating y test
creating X_train
creating y_train
try to return


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(82593, 120, 200, 1)
(82593, 6)
(9176, 120, 200, 1)
(9176, 6)


In [ ]:
del X, y


In [ ]:
# X_train = X_train/255.0
# X_test = X_test/255.0

# y_train = np.asarray(y_train).astype('float32')
# y_test = np.asarray(y_test).astype('float32')


In [ ]:
what_label = y_train
new_dict_key = dict(list(keys_categories.items())[1:-1])
only_nothing = 0
for key_pos, key_name in enumerate(new_dict_key):
    press_number = 0 
    for label in what_label:
        for index, num in enumerate(label):
            if num == 1 and index==key_pos:
                press_number += 1
    print(f'In {len(what_label)} categories. You pressed key with position {key_name}:')
    print(f'{press_number} times')
    print('its have {:.2f} percent'.format(press_number*100/len(what_label)))
    print(f'________________________________________________________________________')
for lable in what_label:
    if not any(lable):
        only_nothing += 1  
    
    
print(f'In {len(what_label)} categories. You pressed do nothing')
print(f'{only_nothing} times')
print('its have {:.2f} percent'.format(only_nothing*100/len(what_label)))
print(f'________________________________________________________________________')


In 82593 categories. You pressed key with position A:
13723 times
its have 16.62 percent
________________________________________________________________________
In 82593 categories. You pressed key with position S:
13767 times
its have 16.67 percent
________________________________________________________________________
In 82593 categories. You pressed key with position D:
13780 times
its have 16.68 percent
________________________________________________________________________
In 82593 categories. You pressed key with position SPACE:
13779 times
its have 16.68 percent
________________________________________________________________________
In 82593 categories. You pressed key with position E:
13756 times
its have 16.66 percent
________________________________________________________________________
In 82593 categories. You pressed key with position SHIFT:
0 times
its have 0.00 percent
________________________________________________________________________
In 82593 categories. You 

In [ ]:
X_train.shape[1:]
y_train.shape[1:]


(6,)

In [ ]:
model = Sequential()
# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=X_train.shape[1:], kernel_size=(11,11),\
strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())
# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())
# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())
# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())
# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())
# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
#  output Layer 
model.add(Dense(len(y_train[0])))
model.add(Activation('sigmoid'))

model.compile(
loss="binary_crossentropy",
optimizer='adam',
metrics=['accuracy']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 48, 96)        11712     
_________________________________________________________________
activation_8 (Activation)    (None, 28, 48, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 24, 96)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 14, 24, 96)        384       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 24, 256)       614656    
_________________________________________________________________
activation_9 (Activation)    (None, 14, 24, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 11, 256)       

In [ ]:
NAME = 'CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard])
model.save(f'{path_gd}{level}/cnn_{level}.h5')